In [1]:
import pandas as pd
import numpy as np
import json
import os
os.chdir("/content/drive/MyDrive/3. Grad School/Lab Meetings/2025.08.29/Instruction Tuning/")

# 모델 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
! pip -q install -U "transformers>=4.43" accelerate bitsandbytes sentencepiece
! pip install -U bitsandbytes --only-binary=:all:

# Instruction Tuning
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.6 MB/s eta 0:00:00


#### 데이터 가공

In [ ]:
### 데이터셋 가져오기
with open("DATA/Financial_Words.json", "r", encoding = "utf-8") as f:
    financial_words = json.load(f)
financial_words[:3]

[{'용어': '휴면예금',
  '의미': '은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다. Dormant Deposit'},
 {'용어': '휴면보험금',
  '의미': '보험계약이 실효되거나 만기되어 보험금이나 환급금 등이 발생하였음에도, 보험계약자가 이를 3년(2015.3.12.부터 3년으로 연장) 동안 찾아가지 않아 소멸시효가 완성되어 보험회사에서 보관하고 있는 것을 의미한다. 보험계약이 실효된 뒤 3년이 경과된 계약의 환급금, 만기가 지난 뒤에도 찾아가지 않은 만기 보험금 등이 여기에 해당한다. 휴면보험금은 청구권이 소멸된 금액으로서 상법상으로는 보험회사에 귀속되나, 당연히 보험계약자에게 돌아가야 할 돈이기 때문에 휴면보험금이 확인될 경우 보험회사는 계약자에게 환급하고 있다. 이를 위해 보험계약자 등이 자신의 휴면보험금을 확인할 수 있도록 「휴면계좌통합조회시스템 」을 설치, 운영(2006.4월)하고 있으며, 2016.12.16.~2017.1월 기간중 휴면재산 보유사실 통지, 온라인 등 비대면안내 및 환급 등의 방식으로 휴면재산 찾아주기 캠페일을 실시(2016.12월)하였다. 한편, 보험회사는 휴면보험금을 서민금융진흥원(「서민의 금융생활 지원에 관한 법률 」시행(2016.9.23.))에 출연하고 동 재단에서 휴면보험금 관리, 환급업무를 담당하고 있다. Dormant insurance'},
 {'용어': '회전결제(리볼빙)',
  '의미': '회원이 

In [ ]:
### 두 가지 형식으로 가공해서 jsonl 만들기
tuning_data = []

for item in financial_words:
    word = item["용어"]
    definition = item["의미"]
    # 형식 1
    tuning_data.append({
        "instruction" : "다음 용어의 정의를 설명해줘",
        "input" : word,
        "output" : definition
    })

    # 형식 2
    tuning_data.append({
        "instruction" : f"{word}란 무엇인가?",
        "input" : "",
        "output" : definition
    })

with open("DATA/Instruction_Tuning_Data.json", "w", encoding = "utf-8") as f:
    (json.dump(tuning_data, f, ensure_ascii = False))

In [ ]:
tuning_data[:3]

[{'instruction': '다음 용어의 정의를 설명해줘',
  'input': '휴면예금',
  'output': '은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다. Dormant Deposit'},
 {'instruction': '휴면예금란 무엇인가?',
  'input': '',
  'output': '은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다. Dormant Deposit'},
 {'instruction': '다음 용어의 정의를 설명해줘',
  'input': '휴면보험금',
  'output': '보험계약이 실효되거나 만기되어 보험금이나 환급금 등이 발생하였음에도, 보험계약자가 이를 3년(2015.3.12.부터 3년으로 연장) 동안 찾아가지 않아 소멸시효가 완성되어 보험회사에서 보관하고 있는 것을 의미한다. 보험계약이 실효된 뒤 3년이 경

#### 모델 불러오기

In [ ]:
model = "kakaocorp/kanana-1.5-2.1b-instruct-2505"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,                # 모델 양자화해서 받아오기
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map = "auto",
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16
)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
### 금융 용어 튜닝 전
messages = [
    {"role": "system", "content": "You are a helpful finantial expert."},
    {"role": "user", "content": "전화금융사기가 뭐야?"}
]

pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)
output = pipe(messages, max_new_tokens = 512, do_sample = True, temperature = 0.8, return_full_text = False)
print(output[0]["generated_text"])

Device set to use cuda:0


좋은 질문입니다!  
“전화금융사기”란,  
금융 관련 내용을 문자, 전화, 혹은 인터넷 메시지로 가장해 피해자를 속인 뒤, 예금, 카드, 계좌 정보 등을 빼내거나, 대출을 해준다거나, 이체를 요구하는 등 금융 피해를 입히는 범죄를 말합니다.

예를 들어,
- “은행에서 대출이 필요하니 계좌 정보를 보내세요.”
- “급히 돈이 필요하니 입금해 주세요.”
- “보안 상의 문제라 자금을 이체해야 합니다.”
- “정부 지원금을 대신 보내드릴게요.”
와 같이 금융 관련 내용을 속여서 피해자를 현혹하는 방식입니다.

### 주요 특징
- 금융기관이나 공공기관을 사칭하여 접근한다.
- 긴급을 강조하거나, 비밀번호, 계좌번호, 카드번호 등 개인정보를 요구한다.
- 피해자가 의심 없이 정보를 입력하거나 이체를 하면, 이미 범죄가 완료된 상태입니다.

### 예방 방법
- 전화, 문자, 메시지로 금융 관련 요청이 오면 반드시 본인 계좌와 비밀번호 등은 절대 입력하지 마세요.
- 금융회사나 공공기관은 절대 개인정보나 계좌정보를 요구하지 않습니다.
- 거래내역과 계좌상태를 정기적으로 확인하세요.
- 금융사기 피해가 의심되면 즉시 해당 금융회사나 경찰(112)에 신고해야 합니다.

궁금한 점이 더 있으시면 언제든 질문해 주세요!


#### Instruction Tuning

1. 데이터셋 불러오기 (Dataset 활용)
2. 프롬프트 템플릿 생성
3. 토크나이징
4. LoRA 설정
5. Trainer 설정
6. 학습 진행

In [ ]:
### 데이터 준비
with open("DATA/Instruction_Tuning_Data.json", "r", encoding = "utf-8") as f:
    tuning_data = json.load(f)

dataset = Dataset.from_list(tuning_data)
dataset = dataset.train_test_split(test_size = 0.1)

In [ ]:
tuning_data[:3]

[{'instruction': '다음 용어의 정의를 설명해줘',
  'input': '휴면예금',
  'output': '은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다. Dormant Deposit'},
 {'instruction': '휴면예금란 무엇인가?',
  'input': '',
  'output': '은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다. Dormant Deposit'},
 {'instruction': '다음 용어의 정의를 설명해줘',
  'input': '휴면보험금',
  'output': '보험계약이 실효되거나 만기되어 보험금이나 환급금 등이 발생하였음에도, 보험계약자가 이를 3년(2015.3.12.부터 3년으로 연장) 동안 찾아가지 않아 소멸시효가 완성되어 보험회사에서 보관하고 있는 것을 의미한다. 보험계약이 실효된 뒤 3년이 경

In [ ]:
### 프롬프트 템플릿 생성
"""
형식 1.
instruction : ~ 설명해줘
input : 용어
output : 정의
=> prompt = "Instruction : ~ 설명해줘 / Input : 용어"

형식 2.
instruction : 용어란 무엇인가?
input : ""
output : 정의
=> prompt = "Instruction : ~ 설명해줘"
"""
def create_prompts(examples):
    full_texts, prompts = [], []
    for instruction, input, output in zip(examples["instruction"], examples["input"], examples["output"]):
        # 형식 1
        if input:
            prompt = f"Instruction: {instruction}\nInput: {input}\nAnswer:"
        # 형식 2
        else:
            prompt = f"Instruction: {instruction}\nAnswer:"

        full_texts.append(prompt + " " + output)
        prompts.append(prompt)

    return full_texts, prompts

In [ ]:
def tokenize(full_texts, prompts):
    # 전체 다 토크나이즈
    tokenized = tokenizer(
        full_texts,
        max_length = 512,
        truncation = True,
        padding = "max_length"
    )

    # 정답 라벨은
    labels = [seq[:] for seq in tokenized['input_ids']]

    # 프롬프트 부분은 -100 마스킹
    for i, prompt in enumerate(prompts):
        prompt_ids = tokenizer(prompt, max_length = 512, truncation = True)["input_ids"]
        mask_len = min(len(prompt_ids), len(labels[i]))
        labels[i][:mask_len] = [-100] * mask_len

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    model.resize_token_embeddings(len(tokenizer))

    tokenized["labels"] = labels
    return tokenized

In [ ]:
def preprocessing(examples):
    full_texts, prompts = create_prompts(examples)
    tokenized = tokenize(full_texts, prompts)
    return tokenized

tokenized_dataset = dataset.map(preprocessing, batched = True, remove_columns = dataset['train'].column_names)

Map:   0%|          | 0/968 [00:00<?, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
lengths = []
for item in tuning_data:
    lengths.append(len(item["output"]))

print(f"가장 긴 응답 : {max(lengths)}")
print(f"가장 짧은 응답 : {min(lengths)}")
print(f"평균 응답 길이 : {np.mean(lengths)}")
print("==========================")
s = pd.Series(lengths)
print(s.describe())
# s.hist(bins = 10)
print("==========================")
count = (s <= 512).sum()
print(f"512에서 자르면 {count / len(s) * 100 :.2f}%의 데이터가 커버 가능합니다.")

가장 긴 응답 : 1150
가장 짧은 응답 : 61
평균 응답 길이 : 380.6635687732342
count    1076.000000
mean      380.663569
std       143.303250
min        61.000000
25%       277.000000
50%       384.500000
75%       463.000000
max      1150.000000
dtype: float64
512에서 자르면 85.87%의 데이터가 커버 가능합니다.


In [ ]:
### LoRA 설정
model = prepare_model_for_kbit_training(model)

model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none", # bias 파라미터는 학습 x
    task_type = "CAUSAL_LM" # Causal Language Modeling (일반 GPT류)
)

model = get_peft_model(model, lora_config)

In [ ]:
### Trainer 설정
training_args = TrainingArguments(
    output_dir = "./Models",
    per_device_train_batch_size = 2, # GPU 하나당 minibatch 크기
    gradient_accumulation_steps = 8, # 여러 step 동안 gradient 누적했다가 한 번에 업데이트
    # 위의 두 개를 곱한 게 batch size = 16
    num_train_epochs = 3,
    learning_rate = 1e-4,

    logging_dir = "./Models/logs",
    logging_steps = 5, # 몇 step마다 로그를 기록할 것인지
    # step = 전체 데이터 / batch size = 960 / 16 = 60
    eval_strategy ="epoch",
    save_strategy = "epoch", # 체크포인트 저장은 epoch마다
    save_total_limit = 2, # 체크포인트 최대 2개까지만 저장
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",

    fp16 = True, # GPU 메모리 절약, 속도 향상
    optim = "paged_adamw_32bit",
    report_to = "tensorboard" # wandb 안 쓸래,,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)],
)

/tmp/ipython-input-4249633047.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
### 학습 진행
trainer.train()
model.save_pretrained("./Models")
tokenizer.save_pretrained("./Models")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.801100,0.786661
2,0.734600,0.752081
3,0.787100,0.744910


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('./Models/tokenizer_config.json',
 './Models/special_tokens_map.json',
 './Models/chat_template.jinja',
 './Models/tokenizer.json')

#### 결과 확인하기

In [4]:
### 모델 불러오기
base_model = "kakaocorp/kanana-1.5-2.1b-instruct-2505"
peft_model_path = "./Models/checkpoint-183"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map = "auto",
    torch_dtype = torch.bfloat16
)
model = PeftModel.from_pretrained(model, peft_model_path)

In [5]:
### 성능 확인
messages = [
    {"role": "system", "content": "You are a helpful finantial expert."},
    {"role": "user", "content": "전화금융사기가 뭐야?"}
]

pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)
output = pipe(messages, max_new_tokens = 512, do_sample = True, temperature = 0.8, return_full_text = False)
print(output[0]["generated_text"])

Device set to use cuda:0


전화금융사기란, 통화, 문자, 인터넷, 대면 등 다양한 방법을 통해 금전을 요구하거나 금융거래정보를 요구하는 범죄를 말합니다. 피해자가 전화로 대화하던 중 금전이나 비밀번호, 계좌비밀번호 등을 알려주면, 이는 범죄자의 자금세탁 또는 사기대출, 바이러스설치, 해킹, 정보유출 등 범죄에 이용되어 피해자가 금전적 손실을 입게 됩니다. 전화금융사기의 대표적인 유형으로는 대출사기(대출빙자사기), 보이스피싱(보이스텔레폰 피싱), 파밍(Pharming), 스미싱(Smishing), 메신저피싱 등이 있습니다.

전화금융사기는 범죄자의 금전세탁, 송금, 자금운용, 계좌개설 등에 이용되어 금융범죄를 조장하는 역할을 하므로, 금융감독원 및 금융분야 수사기관에 적극적으로 신고하여야 합니다.  
전화금융사기 예방을 위한 수칙은 아래와 같습니다.

- 금융회사의 대출, 보험, 예금 등은 공식적인 경로(금융회사 홈페이지, 금융감독원 콜센터, 금융회사 영업점)로 문의  
- 대출빙자, 금전이체, 계좌정보제공, 전자금융거래, 개인정보제공 등 요구할 경우 의심  
- 의심스러운 금융거래는 즉시 금융회사에 문의  
- 보안카드 번호전부, 계좌비밀번호 등 금융거래 정보를 요구할 경우 즉시 금융회사에 알리고 거래를 중지  
- 인터넷에서는 금융회사 공식 홈페이지 외의 인터넷 사이트에서 거래하지 않음  
- 문자, 메신저 등으로 금융거래 관련 정보를 요구하는 경우 즉시 삭제  
- 금융회사가 직접 연락하여 개인정보나 금융거래정보를 요구하면 즉시 금융회사에 문의하여 확인  
- 자녀를 사칭하여 금전, 신용카드정보, 계좌번호 등 금융거래정보를 요구하는 경우는 미성년자일 가능성이 높으므로 경찰에 신고

전화금융사기를 당하지 않도록 항상 경각심을 갖고 금


#### 추후 발전 방향

- 데이터셋 관련..
    - ~란/이란 구분하면 좋을 듯?
- 단순 정의만 포함하지 않고, 조금 더 다양한 패턴을 반영하면 좋을 듯
- 뭐 이 개념이랑 저 개념을 비교해달라고 하거나.. 어떤 게 더 좋은지 등...